In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 5

In [2]:
G_rwm2_5 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_5.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_5)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_5))

MultiDiGraph with 50000 nodes and 133248 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_5 = list(G_rwm2_5.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_5_df = nx.to_pandas_edgelist(G_rwm2_5) 
rwm2_5_df.head()

,source,target,weight
0,12S7NaikVGY1V7iM4qoH5uyHXV3QWQ1tVk,1LjRr9PwS93ddTK6h8rLZU3NFjUvapJC3r,1.0
1,12S7NaikVGY1V7iM4qoH5uyHXV3QWQ1tVk,38qrAYj4ebTpsLFgMv9MDAGkAEfrLuwvrB,1.0
2,3E73wzSKfpTELLUrZGJyU5YtjbxwuHs2vq,3ECj4CGJyJta7chJJt4EqH2ptaEWcgUA2c,1.0
3,3KX39GpXX5pPvEwd1P7Y3duA3hU5f4SMQY,1L4pBeLzSPMCKVBwGFWdyjdgc1UEuR8pdF,1.0
4,3KX39GpXX5pPvEwd1P7Y3duA3hU5f4SMQY,363uos24LJbDyNqA3fR9712dM6gxt66WHT,1.0


In [8]:
# Number of unique input addresses
rwm2_5_df['source'].nunique()

31897

In [9]:
# Number of unique output addresses
rwm2_5_df['target'].nunique()

31226

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_5_df.describe()

,weight
count,133248.000000
mean,0.783381
std,0.509002
min,-69.648093
25%,0.568715
50%,1.000000
75%,1.000000
max,56.159410


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_5).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  282


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.56%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_5.degree()]
degree = [val for (node, val) in G_rwm2_5.degree()]
in_degree = [val for (node, val) in G_rwm2_5.in_degree()]
out_degree = [val for (node, val) in G_rwm2_5.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_5) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,12S7NaikVGY1V7iM4qoH5uyHXV3QWQ1tVk,2,0,2,0
1,3E73wzSKfpTELLUrZGJyU5YtjbxwuHs2vq,3,2,1,0
2,3ALa94jLM3U6UYiUcieez59t6YQBDCgWsz,1,1,0,0
3,3KX39GpXX5pPvEwd1P7Y3duA3hU5f4SMQY,4,2,2,0
4,33TWSUiipHXqYTRZkHjD3f96qu4RoJ4zp9,9,0,9,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.329920,2.664960,2.664960
std,40.083613,17.248836,26.294662
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4384.000000,1500.000000,2896.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,282.000000,282.000000,282.000000
mean,136.223404,55.716312,80.507092
std,483.350462,189.363358,322.358317
min,1.000000,0.000000,0.000000
25%,6.000000,2.000000,1.000000
50%,23.500000,9.000000,7.000000
75%,73.750000,36.750000,30.750000
max,4384.000000,1500.000000,2896.000000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49718.000000,49718.000000,49718.00000
mean,4.587493,2.364053,2.22344
std,14.057599,8.971274,8.56975
min,1.000000,0.000000,0.00000
25%,1.000000,0.000000,0.00000
50%,2.000000,1.000000,1.00000
75%,4.000000,2.000000,2.00000
max,939.000000,934.000000,641.00000


For all 3 cases the degree distribution properties are similar to the overall network.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_5) # Connectivity remains very low

5.330026600532011e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_5 = k_clique_communities(G_rwm2_5.to_undirected(), k=10)
com_rwm2_5 = next(com_generator_rwm2_5)
print('The number of communities in the sample : ' + str(len(com_rwm2_5)))

The number of communities in the sample : 89


The sample has a large variety of community structures. (Note: Giant Component = 107)